In [3]:
#This notebook builds a ensemble from baseline randomforest using 
#Morgan fingerprint 
#and cell painting

In [4]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

In [5]:
list_of_lists_df = pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays.csv")
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.648612,1,0.317810,0.794860,1,0.290626,1.0,0.282609,0.858368,Training,True,True,0.742456,0.855408
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.261878,0,0.317810,0.587383,1,0.290626,1.0,0.576923,0.710612,Training,False,True,0.412004,0.709168
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.025140,0,0.317810,0.180529,0,0.290626,0.0,0.727273,0.492026,Training,True,True,0.039553,0.310587
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.018895,0,0.317810,0.230988,0,0.290626,0.0,0.800000,0.717828,Training,True,True,0.029727,0.397398
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.258888,0,0.317810,0.116859,0,0.290626,0.0,0.738462,0.457042,Training,True,True,0.407301,0.201047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,"InChI=1S/C8H11ClN2O3S2/c1-4(2)16(13,14)5-3-15-...",0.309551,1,0.240921,0.322350,1,0.260921,0.0,0.250000,0.618373,HeldOut,False,False,0.545206,0.541558
58483,93_259,InChI=1S/C18H23NO3/c1-13(12-22-17-6-4-3-5-7-17...,0.267934,1,0.240921,0.223786,0,0.260921,0.0,0.391304,0.603436,HeldOut,False,True,0.517793,0.428839
58484,93_259,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.335675,1,0.240921,0.283991,1,0.260921,1.0,0.277778,0.826193,HeldOut,True,True,0.562413,0.515607
58485,93_259,InChI=1S/C16H13N3O3/c1-22-16(21)19-15-17-12-8-...,0.334415,1,0.240921,0.371684,1,0.260921,0.0,0.344828,0.763229,HeldOut,False,False,0.561583,0.574934


In [6]:
list_of_lists_df["simple_pred"]=round((list_of_lists_df["fp_proba_scaled"] + list_of_lists_df["CP_proba_scaled"])/2)
list_of_lists_df["Ensemble_Correct"] = list_of_lists_df["simple_pred"]==list_of_lists_df["true"]
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.648612,1,0.317810,0.794860,1,0.290626,1.0,0.282609,0.858368,Training,True,True,0.742456,0.855408,1.0,True
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.261878,0,0.317810,0.587383,1,0.290626,1.0,0.576923,0.710612,Training,False,True,0.412004,0.709168,1.0,True
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.025140,0,0.317810,0.180529,0,0.290626,0.0,0.727273,0.492026,Training,True,True,0.039553,0.310587,0.0,True
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.018895,0,0.317810,0.230988,0,0.290626,0.0,0.800000,0.717828,Training,True,True,0.029727,0.397398,0.0,True
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.258888,0,0.317810,0.116859,0,0.290626,0.0,0.738462,0.457042,Training,True,True,0.407301,0.201047,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,"InChI=1S/C8H11ClN2O3S2/c1-4(2)16(13,14)5-3-15-...",0.309551,1,0.240921,0.322350,1,0.260921,0.0,0.250000,0.618373,HeldOut,False,False,0.545206,0.541558,1.0,False
58483,93_259,InChI=1S/C18H23NO3/c1-13(12-22-17-6-4-3-5-7-17...,0.267934,1,0.240921,0.223786,0,0.260921,0.0,0.391304,0.603436,HeldOut,False,True,0.517793,0.428839,0.0,True
58484,93_259,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.335675,1,0.240921,0.283991,1,0.260921,1.0,0.277778,0.826193,HeldOut,True,True,0.562413,0.515607,1.0,True
58485,93_259,InChI=1S/C16H13N3O3/c1-22-16(21)19-15-17-12-8-...,0.334415,1,0.240921,0.371684,1,0.260921,0.0,0.344828,0.763229,HeldOut,False,False,0.561583,0.574934,1.0,False


In [7]:
list_of_lists_df.to_csv("Predictions_train_heldout_scaled_prob_all_assays_ensemble.csv", index=False)